In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
pathname='shared-ns1000k/dataporten-home/b8b982ef-2de983-2d4813-2dbbd2-2d5e4e04927306/'
match_date_file = pathname + 'AMDE_NYA.txt'
date_data = pd.read_csv(match_date_file, header = None,
                   parse_dates = True)
date_data.columns = ['Date/Time']
date_data['Date/Time'] = pd.to_datetime(date_data['Date/Time'])
date_data = date_data.set_index('Date/Time')

In [ ]:
def find_start(file):
    i = 1
    with open(file, 'r') as input:
        for line in input:
            if '*/' in line:
                break
            i = i + 1
    return i

def file_reader(filename):
    skip_rows = find_start(filename)
    data = pd.read_csv(filename, sep='\t', skiprows=skip_rows, 
                   index_col = 'Date/Time',
                   dtype={'DIR [W/m**2]': 'float64',
                          'LWD [W/m**2]': 'float64'},
                   parse_dates = True)
    daily_data=data.resample('D').mean()
    daily_data = daily_data.loc[date_data.index].dropna()
    return daily_data

In [ ]:
path = 'shared-ns1000k/inputs/BSRN_NYA/'
filename = filename = path + 'NYA_radiation_'+ str(date_data.index[0].year) + '-0' + str(date_data.index[0].month) + '.tab'  
daily_data = file_reader(filename)
for i in range(1,date_data.index.shape[0]):
    filename = path + 'NYA_radiation_'+ str(date_data.index[i].year) + '-0' + str(date_data.index[i].month) + '.tab'
    daily_data = daily_data.append(file_reader(filename))
    

In [ ]:
daily_data=daily_data.drop_duplicates(keep='first')